### High level accommplishments:
- Convert gamma params from x_bar, and s^2
- Distributions, Common types
- Categories such as scalar, vector, matrix returning
- Exercise to show these distributions. 

## Day 8 Distributions:


- Sample from some distributions:
    - Salar returning distributions
        - Normal distributions
        - gamma distribution
        - exponential
        - Poisson (discrete)
        - Negative Binomial
        - Geometric
        
    - vector returning distribution, e.g., $p (1 \times)$ dimensional vector
$$ \left[ \begin{array}{c}x_1\\x_2\\ \vdots\\ x_p \end{array}\right] $$    
        - multivariate normal distribution <br><br>

    - matrix returning distribution, e.g., $p (1 \times)$ dimensional vector
$$ \left[ \begin{array}{cccc}x_{11}&x_{12}&\cdots&x_{1p}\\x_{21}&x_{22}&\cdots&\\x_{2p}& \vdots & \ddots & \vdots \\ x_{n1}&x_{n2}&\cdots&x_{np}\end{array}\right] $$    
        - wishart distribution


- marginal distributions (e.g. $X$ where $X-~poisson(\lambda)$
- join distriution -- collection (vector) of random varialbes
- conditional distribution -- conditional idnependence
- i.i.d.

- talked about Poisson distribution...

- now we'll look at some samples from a distribution...


In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import math

from bokeh.io import output_notebook, show, output_file, push_notebook
from bokeh.plotting import figure


# For dynamic interactive data as a dictionary
from bokeh.models import ColumnDataSource

# Widget to accept inputs to modify the graphs. 
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Dropdown, Slider
from ipywidgets import interact
output_notebook()

Loading BokehJS ...

In [2]:
number_RVs = 50
number_repeat_experiments=10000
number_of_binom_trials = 10
chance_of_trial_success = .1
sim_data = stats.binom.rvs(size=[number_repeat_experiments, number_RVs], p=chance_of_trial_success, n=number_of_binom_trials)


In [3]:
def get_ePMF_v2(sample):
    '''returns array of length max-min+1
    which counts how many of each outcome
    (as given by the index) were observed
    '''

    my_min=np.min(sample)
    my_max=np.max(sample)
    
    support = np.arange(my_min, my_max+1)
    my_ePMF = 0*support
    for i, outcome in enumerate(support):
        my_ePMF[i] = np.sum(sample==outcome)
    

    return (support, my_ePMF/sample.size)   

glyph_support, y_counts = get_ePMF_v2(np.array([1,1,1,2,3]))
print (glyph_support, y_counts)

[1 2 3] [ 0.6  0.2  0.2]


In [4]:
#create circle plot with bokeh
#support is arange(0-10)
# support = np.arange(101)
samp = np.arange(51)  
empirical_pmf = get_ePMF_v2(samp)  #stats.binom.cdf(support, n, p)
#the one based on data we have. 

#make bokeh glyph

p = figure(title="iid", plot_height=300, plot_width=600, y_range=[0,.5], x_range=[0,100])
r = p.circle(empirical_pmf[0], empirical_pmf[1], color='blue', line_width=3)

def update(d='poisson', mu=50, var=10, p=.5, n=100):
    
    
    if d=='poisson':
        tmp = get_ePMF_v2(stats.poisson.rvs(size=n, mu=mu))
        
    if d=='binom':
        # mu = E[X] = n*p
        # var = Var[X] = n*p*(1-p)
        p = 1-(var/mu) #var must be less than mu or else the number is greater than 1 and not between .01 and .99
        nn = int(mu/p)
        tmp = get_ePMF_v2(stats.binom.rvs(size=n, p=p, n=nn))

    if d=='neg_binom':
        # mu = pN / (1-p)
        # var = pN/(1-p)^2
        
        p = 1-mu/var #mu must be less than var or else the number is greater than 1 and not between .01 and .99
        nn = int(mu*(1-p)/p)
        tmp = get_ePMF_v2(stats.nbinom.rvs(size=n, p=p, n=nn))

    if d=='geom':
        tmp = get_ePMF_v2(stats.geom.rvs(size=n, p=p))
    
    r.data_source.data['x'] = tmp[0]
    r.data_source.data['y'] = tmp[1]
    push_notebook()

show(p, notebook_handle=True)




In [5]:
interact(update, d=['poisson', 'binom', 'geom', 'neg_binom'], mu=(1,100), var=(1,100), 
         p = (.01, .99, .01), n=(1, 10000))

<function __main__.update>

In [6]:
from scipy import stats

In [7]:


samp = np.arange(51)
grid = np.linspace(0, 100, 10000)
y_over_grid = stats.gaussian_kde(samp, bw_method =.1).evaluate(grid)


p = figure(title="first", plot_height=300, plot_width=600, y_range=[0,1], x_range=(0,100))
r = p.line(grid, y_over_grid, color='blue', line_width=3)
r2 = p.circle(samp, 0*samp, color='red', line_width=3, alpha=.1)

def update(d='gamma',  mu=50, var=10, n=100, bw=1):
    
    if d=='gaussian':
        samp = stats.norm.rvs(size=n, loc=mu, scale= np.sqrt(var))
    
    if d=='gamma':
        beta = mu/var
        alpha_ = var**2/var
        print(alpha_)
        samp= stats.gamma.rvs(support, a=alpha_, scale=1./beta )

    
    if d=='expon':
        samp= stats.expon.rvs(size=n, scale=mu)
        
    y_over_grid = stats.gaussian_kde(samp, bw_method =bw).evaluate(grid)
    r.data_source.data['y'] = y_over_grid
    
    r2.data_source.data['x'] = samp
    r2.data_source.data['y'] = 0*samp

    push_notebook()

show(p, notebook_handle=True)



In [8]:
interact(update, d=['gaussian', 'gamma', 'expon'], mu=(1,100), var=(1,100), 
         n=(1, 10000), bw=(1,20))

<function __main__.update>